# SPI Momentum

Add a description of the project here, just a brief overview. Describe the context, datasets used, and key objectives

### Table of Content
List all other chapters here to facilitate navigation

### Setup and Dependencies
Import Libraries: Create a separate cell for importing all the necessary libraries (e.g., pandas, numpy, matplotlib, etc.).
Import functions: Import all the necessary funcitons for data loading, backtesting, etc.
Set Environment Variable: 
Set cache directory (if needed):

In [1]:
import sys
import os

# Navigate up two levels from the notebooks folder to reach the project root and add `src` to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)
print(project_root)

# Import from src/analysis
from src.analysis import backtest_functions

# Import from src/data_processing
#from src.data_processing import data_processing
from src.data_processing import extract_date_and_data

/Users/steve/Documents/Coding/my-project/Abnormal-Returns-in-the-Danish-Stock-Market-With-Momentum-Strategies
(array([0, 3, 4]), array([2, 3, 5]))
[[0.    ]
 [0.0005]
 [0.0005]
 [0.0145]
 [0.0145]
 [0.0155]]


### Data Import and Overview

Load data here and explain what the processing does

In [3]:
# Load and clean the data
#cleaned_df = data_processing.load_and_clean_data()

# Display the cleaned data
#print(cleaned_df.head())

# Process the date column and extract the rest of the data
df = pd.read_csv(project_root.join("data/interim/interim_data.csv")
formatted_date, data = data_processing.process_dataframe(df)

# Display the formatted date and the rest of the data
print("Formatted Date:", formatted_date.head())
print("Data without Date Column:", data.head())

Loading data...


ValueError: Worksheet named 'Tabelle1' not found

### Set up Basic Parameters for Analysis

Define all the variables: Lookback_period, holding_period, etc.

### Momentum Analysis
Run the analysis code here and compare results to benchmark, show performance stats. etc., show Long and Long-Short + table with most important stats

### Robustness Checks
Perform robustness checks here over different input parameters that we "arbitraily" chose: lookback period, holding period, nLong/nShort, trx cost

### Interpretation and Conclusion 

In [60]:
import os
print(os.getcwd())

/Users/steve/Documents/Coding/my-project/Abnormal-Returns-in-the-Danish-Stock-Market-With-Momentum-Strategies/notebook
